In [1]:
# Initial code was based on one of the commits from this github repo: https://github.com/titu1994/Wide-Residual-Networks
# Multiple changes and fixes have been applied since cloning the repo

import numpy as np
import sklearn.metrics as metrics
import tensorflow as tf
import os, os.path
import pandas as pd
import math

import wide_residual_network_fix_v4 as wrn
from keras.datasets import cifar10
import keras.callbacks as callbacks
import keras.utils.np_utils as kutils
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import plot_model
from keras.optimizers import SGD
from keras.callbacks import LearningRateScheduler

from keras import backend as K
from keras.backend.tensorflow_backend import set_session

Using TensorFlow backend.


KeyboardInterrupt: 

In [2]:
name = 'KaggleV2.3 - WRN-28-10'

if not os.path.exists(name):
    os.makedirs(name)

In [3]:
# GPU Memory limitations
# config = tf.ConfigProto()
# config.gpu_options.per_process_gpu_memory_fraction = 0.8
# set_session(tf.Session(config=config))

In [4]:
lr_schedule = [60, 120, 160] # epoch_step
def schedule(epoch_idx):
    if (epoch_idx + 1) < lr_schedule[0]:
        return 0.1
    elif (epoch_idx + 1) < lr_schedule[1]:
        return 0.02 # lr_decay_ratio = 0.2
    elif (epoch_idx + 1) < lr_schedule[2]:
        return 0.004
    return 0.0008

In [5]:
def random_crop(x, random_crop_size = (32,32), sync_seed=None):
    np.random.seed(sync_seed)
    w, h = x.shape[1], x.shape[2]
    rangew = (w - random_crop_size[0]) // 2
    rangeh = (h - random_crop_size[1]) // 2
    offsetw = 0 if rangew == 0 else np.random.randint(rangew)
    offseth = 0 if rangeh == 0 else np.random.randint(rangeh)
    return x[:, offsetw:offsetw+random_crop_size[0], offseth:offseth+random_crop_size[1]]

In [6]:
def pad(x, pad=4):
    return np.pad(x, ((0,0), (pad,pad),(pad,pad),(0,0)), mode='reflect')

In [7]:
batch_size = 64
nb_epoch = 200
img_rows, img_cols = 32, 32

(trainX, trainY), (testX, testY) = cifar10.load_data()

trainX = pad(trainX)
testX = pad(testX)

trainX = trainX.astype('float32')
trainX = (trainX - [125.3, 123.0, 113.9]) / [63.0, 62.1, 66.7]
testX = testX.astype('float32')
testX = (testX - [125.3, 123.0, 113.9]) / [63.0, 62.1, 66.7]


trainY = kutils.to_categorical(trainY)
testY = kutils.to_categorical(testY)

generator = ImageDataGenerator(zca_epsilon=0,
                               horizontal_flip=True,
                               fill_mode='reflect',)

generator.fit(trainX, seed=0, augment=True)

test_generator = ImageDataGenerator(zca_epsilon=0,
                               horizontal_flip=True,
                               fill_mode='reflect',)

test_generator.fit(testX, seed=0, augment=True)


In [8]:
# For WRN-16-8 put N = 2, k = 8
# For WRN-28-10 put N = 4, k = 10
# For WRN-40-4 put N = 6, k = 4
init_shape = (3, 32, 32) if 0 == 'th' else (32, 32, 3)
model = wrn.create_wide_residual_network(init_shape, nb_classes=10, N=4, k=10, dropout=0.0)

model.summary()
# plot_model(model, "WRN-28-10.png", show_shapes=True)

opt = SGD(lr=0.0008, nesterov=True, decay=0.0005)

model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["acc"])
print("Finished compiling")
print("Allocating GPU memory")

model.load_weights('KaggleV2.3 - WRN-28-10/WRN-28-10 Weights.h5')
print("Model loaded.")

Wide Residual Network-22-10 created.
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 32, 32, 3)     0                                            
____________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D) (None, 34, 34, 3)     0           input_1[0][0]                    
____________________________________________________________________________________________________
conv2d_1 (Conv2D)                (None, 34, 34, 16)    432         zero_padding2d_1[0][0]           
____________________________________________________________________________________________________
batch_normalization_1 (BatchNorm (None, 34, 34, 16)    64          conv2d_1[0][0]                   
______________________________________________________

In [9]:
# Callbacks
csv_logger = callbacks.CSVLogger(name + '/log.csv', append=True, separator=';')
save_c = callbacks.ModelCheckpoint(name + '/WRN-28-10 Weights.h5', monitor="val_acc", save_best_only=True)
lrs = LearningRateScheduler(schedule=schedule)

In [10]:
for i in range(0, nb_epoch):
    print('i = ' + str(i))
    for x_batch, y_batch in generator.flow(trainX, trainY, batch_size=len(trainX)):
        for testx_batch, testy_batch in test_generator.flow(testX, testY, batch_size=len(testX)):
            x_batch = random_crop(x_batch)
            testx_batch = random_crop(testx_batch)
            model.fit(x_batch, y_batch, nb_epoch=1, callbacks=[csv_logger, save_c, lrs], validation_data=(testx_batch, testy_batch))
            break
        break

scores = model.evaluate_generator(test_generator.flow(testX, testY, nb_epoch), (testX.shape[0] / batch_size + 1))
print("Accuracy = %f" % (100 * scores[1]))

i = 0


c:\users\intel\anaconda3\envs\ml\lib\site-packages\ipykernel_launcher.py:7: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  import sys


Train on 50000 samples, validate on 10000 samples
Epoch 1/1
50000/50000 [==============================] - 653s - loss: 0.0059 - acc: 0.9982 - val_loss: 0.3900 - val_acc: 0.9210
i = 1
Train on 50000 samples, validate on 10000 samples
Epoch 1/1
50000/50000 [==============================] - 643s - loss: 0.0050 - acc: 0.9984 - val_loss: 0.4227 - val_acc: 0.9156
i = 2
Train on 50000 samples, validate on 10000 samples
Epoch 1/1
50000/50000 [==============================] - 643s - loss: 0.0033 - acc: 0.9990 - val_loss: 0.3561 - val_acc: 0.9239
i = 3
Train on 50000 samples, validate on 10000 samples
Epoch 1/1
50000/50000 [==============================] - 643s - loss: 0.0024 - acc: 0.9995 - val_loss: 0.3622 - val_acc: 0.9291
i = 4
Train on 50000 samples, validate on 10000 samples
Epoch 1/1
50000/50000 [==============================] - 642s - loss: 0.0064 - acc: 0.9979 - val_loss: 0.3877 - val_acc: 0.9212
i = 5
Train on 50000 samples, validate on 10000 samples
Epoch 1/1
50000/50000 [=======

KeyboardInterrupt: 

In [ ]:
def submitFile(modelFile, saveFile, model):
    # model.load_weights(modelFile)
    test_x = np.load('All images.npy')
    test_x[1] = (test_x[1] - test_x[1].mean(axis=0)) / (test_x[1].std(axis=0))
    bs = 100
    steps = math.ceil(int(test_x.shape[0])/bs)
    print ("Steps - " + str(steps))
    pred = model.predict_generator(test_generator.flow(test_x, batch_size=bs, shuffle=False), steps=steps)
    y_pred = np.argmax(pred, axis=1)
    labels = labelsToStrings(y_pred)
    labels.to_csv(saveFile)

In [ ]:
def labelsToStrings(data):
    ret = []
    for label in data:
        temp = ''
        
        if(label == 0): temp = 'airplane'
        if(label == 1): temp = 'automobile'
        if(label == 2): temp = 'bird'
        if(label == 3): temp = 'cat'
        if(label == 4): temp = 'deer'
        if(label == 5): temp = 'dog'
        if(label == 6): temp = 'frog'
        if(label == 7): temp = 'horse'
        if(label == 8): temp = 'ship'
        if(label == 9): temp = 'truck'
        
        ret.append(temp)
    
    label_id = np.arange(data.shape[0]) + 1
    s = pd.Series(ret, index=label_id)
    s = s.to_frame('label')
    s.index.name='id'
    
    return s

In [ ]:
# submitFile(modelFile=name + '/WRN-28-10 Weights-hand_saved_FULL.h5', saveFile='cifar10_submit_' + name + '.csv', model=model)

In [11]:
model.save(name + '/WRN-28-10 Weights-hand_saved_FULL.h5')